In [1]:
import pandas as pd

In [5]:
from mit_d3m import load_dataset

Using TensorFlow backend.


In [6]:
dataset = 'LL1_ItalyPowerDemand'

In [7]:
ds = load_dataset(dataset)
print(ds.context['entities'].keys())
ds.X.head(5)

Getting file datasets/LL1_ItalyPowerDemand.tar.gz from S3 bucket d3m-data-dai
Extracting data/LL1_ItalyPowerDemand.tar.gz
dict_keys(['learningData', 'timeseries'])


,timeseries_file
d3mIndex,
0,0000_train_ts.csv
1,0001_train_ts.csv
2,0002_train_ts.csv
3,0003_train_ts.csv
4,0004_train_ts.csv


In [8]:
ds.context['entities']['timeseries'][0].head(5)

,timeseries_id,time,value,d3mIndex
0,0,0,-0.71052,0
1,1,1,-1.18330,0
2,2,2,-1.37240,0
3,3,3,-1.59310,0
4,4,4,-1.46700,0


In [55]:
from datetime import datetime, timedelta

start = datetime(2010, 1, 1)
cutoff = datetime(2010, 1, 25)

In [228]:
#train = ds.context['entities']['learningData'][0].copy()
X = ds.X.copy()
X['target'] = ds.y.copy()

X.index.name = 'timeseries_id'
X.reset_index(inplace=True, drop=False)

X.index.name = 'demand_id'
X.reset_index(inplace=True, drop=False)

X['cutoff_time'] = cutoff
X.reset_index(drop=True, inplace=True)

demand = X[['demand_id', 'timeseries_id', 'cutoff_time', 'target']].copy()

print(demand.shape)
demand.head()

(67, 4)


,demand_id,timeseries_id,cutoff_time,target
0,0,0,2010-01-25,1
1,1,1,2010-01-25,1
2,2,2,2010-01-25,2
3,3,3,2010-01-25,2
4,4,4,2010-01-25,1


In [229]:
timeseries = X[['timeseries_id', 'timeseries_file']].copy()
timeseries.head()

,timeseries_id,timeseries_file
0,0,0000_train_ts.csv
1,1,0001_train_ts.csv
2,2,0002_train_ts.csv
3,3,0003_train_ts.csv
4,4,0004_train_ts.csv


In [230]:
data = ds.context['entities']['timeseries'][0].copy()

columns = {
    'time': 'timestamp',
    'timeseries_id': 'data_id',
    'd3mIndex': 'timeseries_id'
}
data.rename(columns=columns, inplace=True)
data['timestamp'] = data.timestamp.apply(lambda d: start + timedelta(days=d))

data = data[['data_id', 'timeseries_id', 'timestamp', 'value']]

print(data.shape)
data.head()

(1608, 4)


,data_id,timeseries_id,timestamp,value
0,0,0,2010-01-01,-0.71052
1,1,0,2010-01-02,-1.18330
2,2,0,2010-01-03,-1.37240
3,3,0,2010-01-04,-1.59310
4,4,0,2010-01-05,-1.46700


In [231]:
demand.to_csv('../datasets/ItalyPowerDemand/demand.csv', index=False)
timeseries.to_csv('.../datasets/ItalyPowerDemand/timeseries.csv', index=False)
data.to_csv('../datasets/ItalyPowerDemand/data.csv', index=False)

In [214]:
meta = {
    "name": "ItalyPowerDemand",
    "tables": [
        {
            "path": "timeseries.csv",
            "name": "timeseries",
            "headers": True,
            "fields": [
                {
                    "name": "timeseries_id",
                    "type": "id",
                    "subtype": "primary"
                },
                {
                    "name": "timeseries_file",
                    "type": "text"
                }
            ],
            "primary_key": "timeseries_id"
        },
        {
            "path": "demand.csv",
            "name": "demand",
            "headers": True,
            "fields": [
                {
                    "name": "demand_id",
                    "type": "id",
                    "subtype": "primary"
                },
                {
                    "name": "timeseries_id",
                    "type": "id",
                    "subtype": "foreign",
                    "ref": {
                        "table": "timeseries",
                        "field": "timeseries_id"
                    }
                },
                {
                    "name": "cutoff_time",
                    "type": "datetime",
                    "subtype": "time_index",
                    "properties": {
                        "format": "%Y-%m-%d"
                    }
                },
                {
                    "name": "target",
                    "type": "categorical",
                    "subtype": "categorical"
                }
            ],
            "primary_key": "demand_id"
        },
        {
            "path": "data.csv",
            "name": "data",
            "headers": True,
            "fields": [
                {
                    "name": "data_id",
                    "type": "id",
                    "subtype": "primary"
                },
                {
                    "name": "timeseries_id",
                    "type": "id",
                    "subtype": "foreign",
                    "ref": {
                        "table": "timeseries",
                        "field": "timeseries_id"
                    }
                },
                {
                    "name": "timestamp",
                    "type": "datetime",
                    "subtype": "time_index",
                    "properties": {
                        "format": "%Y-%m-%d"
                    }
                },
                {
                    "name": "value",
                    "type": "number",
                    "subtype": "float"
                }
            ],
            "primary_key": "data_id"
        }
    ]
}

In [215]:
with open('../datasets/ItalyPowerDemand/metadata.json', 'w') as f:
    json.dump(meta, f, indent=4)

In [263]:
from water.loader import TimeSeriesLoader

loader = TimeSeriesLoader('dataset', 'demand', 'target')

In [264]:
X, y, data = loader.load()

In [265]:
X.head(5)

,demand_id,timeseries_id,cutoff_time
0,0,0,2010-01-25
1,1,1,2010-01-25
2,2,2,2010-01-25
3,3,3,2010-01-25
4,4,4,2010-01-25


In [266]:
y.head(5)

0    1
1    1
2    2
3    2
4    1
Name: target, dtype: int64

In [267]:
data.keys()

dict_keys(['entities', 'relationships', 'target_entity'])

In [268]:
data['entities']['demand'][0].head()

,demand_id,timeseries_id,cutoff_time
0,0,0,2010-01-25
1,1,1,2010-01-25
2,2,2,2010-01-25
3,3,3,2010-01-25
4,4,4,2010-01-25


In [269]:
data['relationships']

[('timeseries', 'timeseries_id', 'demand', 'demand_id'),
 ('timeseries', 'timeseries_id', 'data', 'data_id')]